# Constants and Setup

In [ ]:
import os

# whether to commit and push to git after each optimization. intended for long runs
PUSH_TO_GIT = False  

# parent output directory
EXPERIMENTS_DIR = os.path.join("experiments", "bayes_opt_2")

Optimizer to use. Choose by commenting. (Intent is to use train different ones on differen t machines for efficiency.)

In [ ]:
import torch

# CHOSEN_OPTIMIZER = torch.optim.SGD
CHOSEN_OPTIMIZER = torch.optim.Adam

Number of iterations for Bayesian Optimization

In [ ]:
MAX_ITERATIONS = 10

Models to test for BO. All in the list will be optimized.

In [ ]:
from torchvision import models as tvm
import pretrainedmodels as ptm

# The models we will test
MODELS = (
    ptm.alexnet, # gets maximum recursion limit exceeded exceptions
    ptm.se_resnet50,
    ptm.se_resnet101,
    ptm.inceptionresnetv2,
    ptm.inceptionv4,
    ptm.vgg16,
    ptm.vgg19,
    tvm.resnet101,
    ptm.senet154,
    ptm.nasnetalarge
)

Setup: Make sure Jupyter shows all output

In [ ]:
# show more than one output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# plot charts in our notebook
%matplotlib inline

import subprocess
import sys
sys.setrecursionlimit(1500) # for some reason AlexNet requires more recursive depth

## Helper Functions

In [ ]:
import src.utils as utils
from src.trainable import Trainable
import torch


CRITERION = torch.nn.CrossEntropyLoss() # we'll always use CE for the loss function
 

def train(batch_size, adam_lr, adam_b1, adam_b2, adam_wtdecay):
    """
    Set up a trainable and train it using the given parameters.
    """
    input_params = locals()  # the parameters will become the key
    
    # make an output directory using the model, dataset, and BO iteration
    outdir = make_outdir_name(data_dir, utils.get_model_name_from_fn(chosen_model), 
                              prepend=EXPERIMENTS_DIR,
                              append=str(iteration))
    
    
    image_size = utils.determine_image_size(utils.get_model_name_from_fn(chosen_model))
    dataloaders = utils.get_train_val_dataloaders(
        datadir=data_dir,
        val_proportion=0.15,
        image_size=image_size, 
        batch_size=batch_size
    )
    
    model = build_model(chosen_model)
    utils.fit_model_last_to_dataset(model, dataloaders['train'].dataset)
    
    optimizer = torch.optim.Adam(model.parameters(), 
                                 adam_lr, (adam_b1, adam_b2), adam_wtdecay)
    
    trainable = Trainable(dataloaders, model, CRITERION, optimizer, outdir=outdir)
    acc = trainable.train(50, early_stop_limit=6, verbose=False)
    
    # store this train iteration in a dictionary for later
    global trainables
    key = make_key_from_params(input_params)
    trainables.append({key: trainable})
    return acc
    
    
def build_model(model_fn):
    """
    Build a pretrained model class from a model function. Passes 
    in the appropriate pretrained arg based on the model function's 
    parent module.
    """
    print(model_fn)
    if 'pretrainedmodels' in model_fn.__module__:
        model = model_fn(num_classes=1000, pretrained='imagenet')
    else:
        model = model_fn(pretrained=True)
    return model


def make_key_from_params(params):
    """
    Makes a unique key (as a tuple) from a given list of parameters.
    For storing associated Trainable objects.
    
    """
    return tuple(round(param, 10) for param in params)


def make_outdir_name(datadir, model_name, prepend="", append=""):
    """
    Make the output directory name based on dataset, model name, and any extra info.
    """
    dataset_name = os.path.basename(datadir)
    return os.path.join(prepend, dataset_name, model_name, append) 


# Bayesian Optimization

## Define the Problem

### Domain

In [ ]:
BASE_DOMAIN = [{'name': 'batch_size', 'type': 'discrete', 'domain': (16, 24, 32, 48, 64)},]

ADAM_DOMAIN = BASE_DOMAIN + [
    {'name': 'adam_lr', 'type': 'continuous', 'domain': (0.001, 0.1)},
    {'name': 'adam_beta1', 'type': 'continuous', 'domain': (0.8, .99)},
    {'name': 'adam_beta2', 'type': 'continuous', 'domain': (0.95, .9999)},
    {'name': 'adam_wtdecay', 'type': 'continuous', 'domain': (0, 1)}
]
# TODO: have to figure out how to set a startingdefault
#default_input = [32, 0.001, 0.9, 0.999, 0] 

SGD_DOMAIN = BASE_DOMAIN + [
    {'name': 'lr', 'type': 'continuous', 'domain': (0.001, 0.1)},
    {'name': 'momentum', 'type': 'continuous', 'domain': (0.5, .99)},
    {'name': 'weight_decay', 'type': 'continuous', 'domain': (0, 1)}
]

### Function to optimize

In [ ]:
def f(x):
    """ Value function to maximize for bayesian optimization """
    batch_size = int(x[:, 0])  # the first arg is always batch size
    args = x[:, 1:]  # the remaining depend on if we're using Adam or SGD
    
    val_acc = train(batch_size=batch_size, *args)
    
    return val_acc

## Do BO on all models on both datasets.

In [ ]:
from GPyOpt.methods import BayesianOptimization
from predict import create_predictions
from metrics import create_all_metrics

In [ ]:
def reset_globals(data_dir):
    global iteration  # keep track of our optimization iterations for directory output
    iteration = 0   # but reset to 0 each train run
    global data_dir
    data_dir = data_dir
    
    # reset the global trainables produced by BO
    global trainables
    trainables = {}

In [ ]:
def perform_bayesian_optimization():
    """
    Construct the problem and run the optimization.
    """
    domain = ADAM_DOMAIN if CHOSEN_OPTIMIZER is torch.optim.Adam else SGD_DOMAIN
    problem = BayesianOptimization(
        f=f,
        domain=domain,
        maximize=True
    )
    problem.run_optimization(max_iter=MAX_ITERATIONS)
    return problem

In [ ]:
def plot_bo_results(problem):
    """
    Graph the acquisition function and convergence
    """
    problem.plot_acquisition()
    problem.plot_convergence()

In [ ]:
def get_best_trainable(problem):
    """
    Get the best trainable, based on the optimized parameters, from the global 
    stored trainables.
    """
    best_params = problem.x_opt
    key = make_key_from_params(best_params)
    best_trainable = trainables[key]
    return best_trainable
    

In [ ]:
def generate_test_metrics(trainable):
    """
    Create an itemized predictions file and metrics for the test set.
    """
    predictions_file = create_predictions(
        outdir=trainable.outdir,
        subset='test',
        data_dir=data_dir,
        model=best_trainable.model
    )
    create_all_metrics(predictions_file, trainable.outdir, 'test')

In [ ]:
for model_fn in MODELS:  # iterate over all models
    # set the model
    global chosen_model
    chosen_model = model_fn
    
    # iterate over both binary and quaternary datasets
    for data_index, data_dir in enumerate(
        (os.path.join('data/die_vs_all_tt'), 
        os.path.join('data/4_class_tt'))
    ):
        reset_globals(data_dir)  # reset some globals used for iteration tracking
        
        try:
            # define and optimize the problem
            optimized = perform_bayesian_optimization()
            # plot the results
            plot_bo_results(optimized)
            # get and save the best trainable
            best_trainable = get_best_trainable(optimized)
            best_trainable.save()
            # evalute on the test set using the best model
            generate_test_metrics(best_trainable)
            
        # if something bad happens, skip it so we can let the others run
        except Exception as e:
            print('Skipping because', e)
#             import traceback
#             traceback.print_exc()
            continue
            
        # commit & push only if we can connect to internet
        if PUSH_TO_GIT:
            subprocess.check_call(['git', 'add', 'experiments'])
            subprocess.check_call(['git', 'commit', '-am', f'Results from {str(chosen_model).split()[1]} {data_dir}'])
            subprocess.check_call(['git', 'push'])

# Final Commit and Push

In [ ]:
if PUSH_TO_GIT:
    import time
    time.sleep(120) # wait for two minutes to let everything rendering
    _ = subprocess.check_call(["spd-say", "Your code has finished running"])
    _ = subprocess.check_call(['git', 'commit', '-am', "BO final commit"])
    _ = subprocess.check_call(['git', 'push'])
    